<center>
    <img src="./images/mlfasp.png">
</center>

#### Prof. Dr. -Ing. Gerald Schuller <br> Jupyter Notebook: Renato Profeta

[Applied Media Systems Group](https://www.tu-ilmenau.de/en/applied-media-systems-group/) <br>
[Technische Universität Ilmenau](https://www.tu-ilmenau.de/)

# Neural Networks Basics, Detector

In [1]:
%%html
<iframe width="560" height="315" src="https://www.youtube.com/embed/9ueXaEbRFLY" frameborder="0" allow="accelerometer; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>

The simplest layer is the so-called **fully connected layer.** If we have a 1-dimensional input array "*x*" of length "*in_features*", and a 1-dimensional output array "*y*" of length "*out_features*", then its function is simply a **matrix multiplication** with an addition of an array "*b*", called **"bias"**, followed by a non-linear function, called the **"activation function"**.

In Pytorch the matrix multiplication and addition is the function:

`torch.nn.Linear(in_features, out_features, bias=True)`

The Linear layer computes the function $y=x \cdot A^T + b$, where $.^T$ is the transpose operator for the matrix. The
coefficients of the matrix *A* and the array *b* are called **"weight"**. They will be obtained using optimization, also called **"training"**.

In [1]:
%%html
<iframe width="560" height="315" src="https://www.youtube.com/embed/wKwEfg8eKNA?rel=0" frameborder="0" allow="accelerometer; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>

In [1]:
%%html
<iframe src="https://pytorch.org/docs/master/generated/torch.nn.Linear.html" width="900" height="600"></iframe>

This is called **"fully connected"** because the matrix *A* connects each input element (also called **"feature”**) to each output element or feature.

The input and output arrays are 1-dimensional because of the full and fixed connectivity. If we have an **image** as input we would just **reshape into a 1-dimensional array**, for instance with "*view*" or "*reshape*".

Often neural networks are used as **detectors**. In that case, each value in the output array y would correspond to the output of one detector, and each detector represents one **"class"**.

This linear layer is usually followed by a non-linear function, also called an **"activation function"**, for instance the rectified linear unit function, which is applied to each output element or feature:

`torch.nn.ReLU(inplace=False)`

Applies the element-wise function:

ReLU(x)=max(0,x)

This function limits the output to the non-negative range. This is useful when the output represents a detector. 

In this case, a value of 1 might mean "detected with certainty", and a value of 0 would mean "not detected with certainty".
Here, negative values would make no sense. This is probably the most often used activation function because of its simplicity.

In [2]:
%%html
<iframe src="https://pytorch.org/docs/master/generated/torch.nn.ReLU.html" width="900" height="600"></iframe>

Often a slightly modified version is used, which avoids the vanishing gradient for negative values, which helps the optimization. It has a small slope for negative values, and is hence called “LeakyReLU”:

`torch.nn.LeakyReLU(negative_slope=0.01, inplace=False)`

Applies the element-wise function:

LeakyReLU(x)=max(0,x)+negative_slope∗min(0,x)

In [3]:
%%html
<iframe src="https://pytorch.org/docs/master/generated/torch.nn.LeakyReLU.html" width="900" height="600"></iframe>

The Softmax activation function turns the outputs of a network into looking like a probability function, positive and summing up to 1.

`torch.nn.Softmax(dim=None)`

Softmax is defined as:

$$\large
\text{Softmax}(x_i) = \dfrac{e^x_i}{\sum_j e^x_j}$$

In [4]:
%%html
<iframe src="https://pytorch.org/docs/master/generated/torch.nn.Softmax.html" width="900" height="600"></iframe>

An alternative is the so-called "*sigmoid*" function, which is differential everywhere, but is more complex to compute.
This is the classic activation function which is already used in early papers about neural networks:

`torch.nn.Sigmoid`

Defined as:

$$\large
\text{Sigmoid}(x) = \dfrac{1}{1+e^{-x}}$$

In [5]:
%%html
<iframe src="https://pytorch.org/docs/master/generated/torch.nn.Sigmoid.html" width="900" height="600"></iframe>

## Python Example for a Linear Layer

In [2]:
%%html
<iframe width="560" height="315" src="https://www.youtube.com/embed/XGfYLfcJIsk?rel=0" frameborder="0" allow="accelerometer; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>

For linear layer the shape of the input and output tensors are:
- Input:  (N,∗,Hin) where ∗ means any number of additional dimensions and Hin=in_features.
- Output: (N,∗,Hout) where all but the last dimension are the same shape as the input and Hout=out_features.

N is a **batch size**, * is usually the index for the test set, Hin is the length of our input signal, and Hout is the number of classes (or detectors) that we desire.

The **batch index** is for different sets of training and target data. Optimization is performed one batch after the other. This saves memory because only one batch is loaded into memory.

*Example:*

Imagine we have 2 input signals we want to detect, one increasing and one decreasing:

$$\large
x_0=[1,2], \hspace{1cm} x_1=[2,1]$$

We assemble the 2 signals into a 2-dimensional tensor (a matrix), where each signal is one row:

$$\large
x=
\left[ 
    \begin{array}{cc}
    1 & 2 \\
    2 & 1 \\
    \end{array}
\right]
$$

We want to have 2 detectors, one for the first signal, and one for the second signal. The desired output of the first dectector should be "1" for the first signal $x_0$ and "0" for the second $x_1$. The second detector should output "0" for the first signal $x_0$ and "1" for the second signal $x_1$. The desired output is called the **"target"**.

We assemble the desired outputs or target in a matrix where each detector corrsponds to one row:

$$\large
y=
\left[ 
    \begin{array}{cc}
    1 & 0 \\
    0 & 1 \\
    \end{array}
\right]
$$

For our linear layer we now need to find the weights (coefficients) of the matrix *A* and bias *b* such that the desired output *y* is approximated, given out "*training set*" *x*:

$$\large
y=x \cdot A^T+\mathbf{b}$$

In this simple case we can actually find a closed form solution:

$$\large
b=[0,0], \hspace{1cm} A^T=x^{-1} \cdot y$$

If we now have **more samples in our training set** for *x* and *y*, we will get **tall matrices**. For instance, we could add another input $x_2=[1,1]$ for which both outputs should be zero as target.

For these non-quadratic matrices we can compute the pseudo-inverse, which **minimizes the mean squared error** between the network output and the desired output.

$$\large
y=x\cdot A \\
\large
x^T \cdot y = x^T\cdot x \cdot A \\
\large
(x^T \cdot x)^{-1} \cdot x^T \cdot y = A
$$

**Observe** that this does not include the bias array *b*, which might lead to better approximations, and also needs the mean squared error. Depending on the application, we might want to use a different error measure, also called the **"Loss function"**

A simple and widely used loss function is the mentioned **mean squared error loss**. In Pytorch it is the function:

`torch.nn.MSELoss(reduction='mean')`

For detectors of different classes, usually the so-called **"cross-entropy loss"** is used. It creates a distance or divergence function between two probability distributions.

Here the probability distributions are the target distribution (probability 1 for the true classe, and probability 0 for the false class), and the distribution that the neural network generates (a predicted probability for each class, with high probability value for the class that the network "thinks" is at its input).

In PyTorch:

`torch.nn.CrossEntropyLoss(weight=None, ignore_index=-100, reduction='mean')`

To obtain a better solution and include other loss function, in general **numerical optimization** is used to find the weights of coefficients which minimizes a given loss function. 

**Observe** that the loss function produces a single number or value for a given training set and given weights. The weights are then updated during the optimization to reduce the loss function.

Commom optimizers are **"Stochastic Gradient Descent" (SGD)**, and **"Adaptive Moments” (adam)**. 

After we obtained the weights from the optimization we have to see if it generalizes to other examples, or just works on the training set. For this we have the **validation set**, containing new input and target samples. This is used to modify the neural network structure until it performs on new samples or examples sufficiently.

Once we have a good neural network structure, we test it on the **test set**, another set with new samples or examples, to test the performance.

## Python Example Program

We start our program with importing the Pytorch library and setting the device, where we select "cuda" only if we have a compatible GPU:

In [1]:
import torch
import torch.nn as nn
device='cpu'
#device='cuda'

Then we define our neural network with a linear layer, and a non-linear activation function, which we can comment out on the "forward" function if we want to test a network without the activation function. This is done as a "class", such that the network is an object which can instantiate and initialize in the main part of the program:

In [8]:
%%html
<iframe width="560" height="315" src="https://www.youtube.com/embed/oGAVlqhJkQ8?rel=0" frameborder="0" allow="accelerometer; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>

### Experiment 1

In [2]:
class LinNet(nn.Module):
    def __init__(self):
        super(LinNet, self).__init__()
        # Define the model.
        self.layer1 = nn.Sequential(nn.Linear(in_features=2, out_features=2, bias=True))
        #https://pytorch.org/docs/stable/nn.html?highlight=linear#torch.nn.Linear
        # Generate a fully connected linear neural network model, 1 layer, bias, linear activation function
        # returns: Trainable object
        #self.act = nn.LeakyReLU() #non-linear activation function
        #self.act = nn.ReLU() #non-linear activation function
    
    def forward(self, x):
        out = self.layer1(x)
        #out = self.act(out) #comment out if not desired
        return out

In the main part of the program we generate the data for training and validation:

In [3]:
#input tensor, type torch tensor:
#Indices: batch, sample, features or signal dimension. Here: 1 batch, 3 samples, signal dimension 2:

#Training set:
X=torch.tensor([[1., 2.], [2., 1.],[1., 1.]]).view(1,3,2) #adding the first dimension for the batch
print("X.shape", X.shape)

#Target:
Y=torch.tensor([[1., 0.], [0., 1.],[0., 0.]]).view(1,3,2)
print("Y.shape", Y.shape)

#Validation set, to test generalization:
Xval=torch.tensor([[0.5, 1.0], [1., 0.5],[0.5, 0.5]]).view(1,3,2)
#Validation Target:
Yval=torch.tensor([[1., 0.], [0., 1.],[0., 0.]]).view(1,3,2)

X.shape torch.Size([1, 3, 2])
Y.shape torch.Size([1, 3, 2])


Observe that the validation input data contains a scaled version of the training set.

Now we instantiate the model, define the loss function and the optimizer:

In [4]:
#create network object:
model = LinNet().to(device)
loss_fn = nn.MSELoss()
print("Define loss function:", loss_fn)
#learning_rate = 1e-4
#optimizer = torch.optim.Adam(model.parameters())
optimizer = torch.optim.SGD(model.parameters(),lr=0.1)
print("Define optimizer:", optimizer)

Define loss function: MSELoss()
Define optimizer: SGD (
Parameter Group 0
    dampening: 0
    lr: 0.1
    momentum: 0
    nesterov: False
    weight_decay: 0
)


Here we have the choice of 2 optimizers, Adam and Stochastic Gradient Descent (SGD). We can try which works better.

Next we let the optimizer do 10000 update iterations (called **epochs**):

In [5]:
for epoch in range(10000):
    Ypred=model(X) #the model produces prediction output
    loss=loss_fn(Ypred, Y) #prediction and target compared by loss
    if epoch%100==0:
        print(epoch, loss.item()) #print current loss value
    optimizer.zero_grad() #optimizer sets previous gradients to zero
    loss.backward() #optimizer computes new gradients
    optimizer.step() #optimizer updates weights

0 1.1859253644943237
100 0.03673030063509941
200 0.022693032398819923
300 0.014052987098693848
400 0.008702552877366543
500 0.005389204248785973
600 0.0033373564947396517
700 0.0020667125936597586
800 0.0012798466486856341
900 0.0007925659883767366
1000 0.000490809790790081
1100 0.00030394268105737865
1200 0.00018822094716597348
1300 0.00011655897833406925
1400 7.218079554149881e-05
1500 4.469896884984337e-05
1600 2.768071135506034e-05
1700 1.7141588614322245e-05
1800 1.0615190149110276e-05
1900 6.573673090315424e-06
2000 4.070792783750221e-06
2100 2.521037231417722e-06
2200 1.5611514072588761e-06
2300 9.6681196737336e-07
2400 5.987678264318674e-07
2500 3.708317990458454e-07
2600 2.2966553103742626e-07
2700 1.4223876121377543e-07
2800 8.810457785557446e-08
2900 5.4583626507564986e-08
3000 3.382944413488076e-08
3100 2.0961701707733482e-08
3200 1.2989634257110083e-08
3300 8.04916311381021e-09
3400 4.991105750917768e-09
3500 3.09521719366046e-09
3600 1.9206336521193634e-09
3700 1.19262699

Finally we print out the results on the training set and the validation set, and the obtained weights:

In [6]:
Ypred=model(X) # Make Predictions based on the obtained weights
print("Ypred training set=", Ypred)
loss=loss_fn(Ypred, Y)
print("Loss on trainig set:", loss)
Yvalpred=model(Xval) # Make Predictions based on the obtained weights
print("Y validation set=", Yvalpred)
loss=loss_fn(Yvalpred, Yval)
print("Loss on validation set:", loss)
weights = model.state_dict() #read obtained weights
print("weights=", weights)

Ypred training set= tensor([[[ 1.0000e+00, -8.9407e-07],
         [-8.3447e-07,  1.0000e+00],
         [ 3.2783e-06,  3.2783e-06]]], grad_fn=<AddBackward0>)
Loss on trainig set: tensor(4.6321e-12, grad_fn=<MseLossBackward>)
Y validation set= tensor([[[ 5.3644e-06, -4.9999e-01],
         [-4.9999e-01,  5.3644e-06],
         [-4.9999e-01, -4.9999e-01]]], grad_fn=<AddBackward0>)
Loss on validation set: tensor(0.5000, grad_fn=<MseLossBackward>)
weights= OrderedDict([('layer1.0.weight', tensor([[-4.1555e-06,  1.0000e+00],
        [ 1.0000e+00, -4.1593e-06]])), ('layer1.0.bias', tensor([-1.0000, -1.0000]))])


Each time we execute it, it starts with a different random initialization, hence each time we get a different route the optimization takes, and possibly slightly different results of the optimization. If we get very different results that is a sign that the optimization got stuck in a local minimum.

**Observe**:
We obtain the **desired output on the training set**, but the output for the validation set does not look good. This is reflected by the loss values: Only 4.6493e-12 on the training set, **but 0.5000 on the validation set, a big loss!** Hence we have a bad generalization. 

### Experiment 2

Next we need to modify our neural network structure to try to obtain a better generalization. We do that by adding the activation function, by un-commenting the LeakyReLU function, and the “out=self.act(out)” line:

In [9]:
%%html
<iframe width="560" height="315" src="https://www.youtube.com/embed/aGWzIZu5k70?rel=0" frameborder="0" allow="accelerometer; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>

In [15]:
class LinNet(nn.Module):
    def __init__(self):
        super(LinNet, self).__init__()
        # Define the model.
        self.layer1 = nn.Sequential(nn.Linear(in_features=2, out_features=2, bias=True))
        #https://pytorch.org/docs/stable/nn.html?highlight=linear#torch.nn.Linear
        # Generate a fully connected linear neural network model, 1 layer, bias, linear activation function
        # returns: Trainable object
        self.act = nn.LeakyReLU() #non-linear activation function
        #self.act = nn.ReLU() #non-linear activation function
    
    def forward(self, x):
        out = self.layer1(x)
        out = self.act(out) #comment out if not desired
        return out

In [16]:
#create network object:
model = LinNet().to(device)
loss_fn = nn.MSELoss()
print("Define loss function:", loss_fn)
#learning_rate = 1e-4
#optimizer = torch.optim.Adam(model.parameters())
optimizer = torch.optim.SGD(model.parameters(),lr=0.1)
print("Define optimizer:", optimizer)

Define loss function: MSELoss()
Define optimizer: SGD (
Parameter Group 0
    dampening: 0
    lr: 0.1
    momentum: 0
    nesterov: False
    weight_decay: 0
)


In [17]:
for epoch in range(10000):
    Ypred=model(X) #the model produces prediction output
    loss=loss_fn(Ypred, Y) #prediction and target compared by loss
    if epoch%1000==0:
        print(epoch, loss.item()) #print current loss value
    optimizer.zero_grad() #optimizer sets previous gradients to zero
    loss.backward() #optimizer computes new gradients
    optimizer.step() #optimizer updates weights

0 0.2171287089586258
1000 0.16697926819324493
2000 1.859538497228641e-05
3000 1.8481365259503946e-05
4000 1.8419650587020442e-05
5000 1.8358035958954133e-05
6000 1.829653228924144e-05
7000 1.8235181414638646e-05
8000 1.8174223441747017e-05
9000 1.811349648050964e-05


In [18]:
Ypred=model(X) # Make Predictions based on the obtained weights
print("Ypred training set=", Ypred)
loss=loss_fn(Ypred, Y)
print("Loss on trainig set:", loss)
Yvalpred=model(Xval) # Make Predictions based on the obtained weights
print("Y validation set=", Yvalpred)
loss=loss_fn(Yvalpred, Yval)
print("Loss on validation set:", loss)
weights = model.state_dict() #read obtained weights
print("weights=", weights)

Ypred training set= tensor([[[ 9.9998e-01, -9.2057e-03],
         [-4.8503e-03,  9.9995e-01],
         [ 9.7573e-05,  1.8577e-04]]], grad_fn=<LeakyReluBackward0>)
Loss on trainig set: tensor(1.8053e-05, grad_fn=<MseLossBackward>)
Y validation set= tensor([[[ 2.4266e-01, -4.9970e-03],
         [-4.9984e-03,  4.6056e-01],
         [-2.5728e-03, -3.9320e-04]]], grad_fn=<LeakyReluBackward0>)
Loss on validation set: tensor(0.1441, grad_fn=<MseLossBackward>)
weights= OrderedDict([('layer1.0.weight', tensor([[-0.4851,  0.9999],
        [ 0.9998, -0.9208]])), ('layer1.0.bias', tensor([-0.5146, -0.0788]))])


**Observe**:

There we get a loss on the training set of 0.0000. For the validation set we get 0.1441, indeed much better than before. So using the activation function helps for better generalization.

### Experiment 3

To see the effect of vanishing gradients on the optimization, we uncomment the line with "self.act=nn.ReLU()". This activation function has a constant "0" for negative values and hence a vanishing gradient for negative values.

In [11]:
class LinNet(nn.Module):
    def __init__(self):
        super(LinNet, self).__init__()
        # Define the model.
        self.layer1 = nn.Sequential(nn.Linear(in_features=2, out_features=2, bias=True))
        #https://pytorch.org/docs/stable/nn.html?highlight=linear#torch.nn.Linear
        # Generate a fully connected linear neural network model, 1 layer, bias, linear activation function
        # returns: Trainable object
        #self.act = nn.LeakyReLU() #non-linear activation function
        self.act = nn.ReLU() #non-linear activation function
    
    def forward(self, x):
        out = self.layer1(x)
        out = self.act(out) #comment out if not desired
        return out

In [12]:
#create network object:
model = LinNet().to(device)
loss_fn = nn.MSELoss()
print("Define loss function:", loss_fn)
#learning_rate = 1e-4
#optimizer = torch.optim.Adam(model.parameters())
optimizer = torch.optim.SGD(model.parameters(),lr=0.1)
print("Define optimizer:", optimizer)

Define loss function: MSELoss()
Define optimizer: SGD (
Parameter Group 0
    dampening: 0
    lr: 0.1
    momentum: 0
    nesterov: False
    weight_decay: 0
)


In [13]:
for epoch in range(10000):
    Ypred=model(X) #the model produces prediction output
    loss=loss_fn(Ypred, Y) #prediction and target compared by loss
    if epoch%1000==0:
        print(epoch, loss.item()) #print current loss value
    optimizer.zero_grad() #optimizer sets previous gradients to zero
    loss.backward() #optimizer computes new gradients
    optimizer.step() #optimizer updates weights

0 0.3374039828777313
1000 0.1666666716337204
2000 0.1666666716337204
3000 0.1666666716337204
4000 0.1666666716337204
5000 0.1666666716337204
6000 0.1666666716337204
7000 0.1666666716337204
8000 0.1666666716337204
9000 0.1666666716337204


In [14]:
Ypred=model(X) # Make Predictions based on the obtained weights
print("Ypred training set=", Ypred)
loss=loss_fn(Ypred, Y)
print("Loss on trainig set:", loss)
Yvalpred=model(Xval) # Make Predictions based on the obtained weights
print("Y validation set=", Yvalpred)
loss=loss_fn(Yvalpred, Yval)
print("Loss on validation set:", loss)
weights = model.state_dict() #read obtained weights
print("weights=", weights)

Ypred training set= tensor([[[0.0000e+00, 0.0000e+00],
         [0.0000e+00, 1.0000e+00],
         [0.0000e+00, 1.7285e-06]]], grad_fn=<ReluBackward0>)
Loss on trainig set: tensor(0.1667, grad_fn=<MseLossBackward>)
Y validation set= tensor([[[0.0000, 0.0000],
         [0.0000, 0.2029],
         [0.0512, 0.0000]]], grad_fn=<ReluBackward0>)
Loss on validation set: tensor(0.2730, grad_fn=<MseLossBackward>)
weights= OrderedDict([('layer1.0.weight', tensor([[-0.3343, -0.4287],
        [ 1.0000, -0.4059]])), ('layer1.0.bias', tensor([ 0.4327, -0.5941]))])


**Observe:**

The optimizer gets stuck at a loss value of 0.166... during training. This is cause by the vanishing gradient of the ReLU function in the negative input range.